# YOLO Ojbect detection on video

use pretrained model from COCO Dataset

The code inputs a video file and output the video with object detection. 

In [1]:
import numpy as np
import argparse
import imutils
import time
import cv2
import os

In [24]:

image_path = "./images/"
yolo_path = "./yolo-coco/"
video_path = "./videos/"
out_path = "./output/"

confidence_threshold = 0.5 # prediction confidence threshold
nms_threshold = 0.3 # non-maxima suppression threshold 

In [3]:

labelsPath = os.path.sep.join([yolo_path, "coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")


# use random colors to represent each class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")

# paths to the YOLO weights and model configuration
weightsPath = os.path.sep.join([yolo_path, "yolov3.weights"])
configPath = os.path.sep.join([yolo_path, "yolov3.cfg"])

# load YOLO object detector trained on COCO through cv2
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
print("YOLO loaded")

YOLO loaded


In [4]:
# get only the *output* layer names for prediction
ln = net.getLayerNames()

ln_out = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
print(ln_out)

['yolo_82', 'yolo_94', 'yolo_106']


In [10]:
input_path = video_path+"babbling.mp4"
output_path = out_path + "babbling_output.mp4"

In [28]:

# initialize the video stream, pointer to output video file, and
# frame dimensions
vs = cv2.VideoCapture(input_path)

writer = None
(W, H) = (None, None)
 
# determine the total number of frames in the video file
try:
    prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
        else cv2.CAP_PROP_FRAME_COUNT
    total = int(vs.get(prop))
    print("{} total frames in video".format(total))
 
except:
    print("Could not determine # of frames in video")
    print("No approx. completion time can be provided")
    total = -1

250 total frames in video


# Prediction and output

Loop over frames from the video file stream, process it frame by frame

Make prediction -> draw the predicted box, class, confidence on the image -> write to output

In [29]:

while True:
    # read the next frame from the file
    (grabbed, frame) = vs.read()

    # if grabbed is false, then we have reached the end of the stream
    if not grabbed:
        break

    # save the H and W of the image and scale back the image later
    if W is None or H is None:
        (H, W) = frame.shape[:2]
        
    # construct a blob from the input frame resize it to 416*416 
    # and then make a prediction, results are bounding boxes
    # and associated probabilities
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
        swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(ln_out)
    end = time.time()
    
    
    boxes = []  # bounding boxes
    confidences = [] # confidences
    classIDs = [] # class type

    # loop over each of the layer outputs ['yolo_82', 'yolo_94', 'yolo_106']

    for output in layerOutputs:
        # loop over each of the detections 
        confidence_list = np.array([detection[5:][np.argmax(detection[5:])] for detection in output])

        # filter out weak predictions

        for detection in np.array(output)[confidence_list>confidence_threshold]:

            # extract the class ID and confidence (i.e., probability)
            # of the current object detection
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            
 
            # YOLO returns the center (x, y)-coordinates of
            # the bounding box followed by the boxes' width and
            # height
            box = detection[0:4] * np.array([W, H, W, H])
            (centerX, centerY, width, height) = box.astype("int")

            # get top and and left corner of the bounding box
            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))

            # update list of bounding box coordinates, confidences, and class IDs
            boxes.append([x, y, int(width), int(height)])
            confidences.append(float(confidence))
            classIDs.append(classID)

    # non-maxima suppression 
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold,
        nms_threshold)

    # if there is detection
    if len(idxs) > 0:
        for i in idxs.flatten():
            # bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            # draw a bounding box rectangle and label on the frame
            color = [int(c) for c in COLORS[classIDs[i]]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(LABELS[classIDs[i]],
                confidences[i])
            cv2.putText(frame, text, (x, y - 5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    if writer is None:
        # initialize our video writer
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(output_path, fourcc, 30,
            (frame.shape[1], frame.shape[0]), True)

    # write the output frame to disk
    writer.write(frame)
    
print("Done...")
writer.release()
vs.release()    

Done...
